<a href="https://colab.research.google.com/github/hoiyurie/tibame-ml0602/blob/main/titanic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
from urllib.request import urlretrieve
url = "https://github.com/hoiyurie/tibame-ml0602/raw/main/titanic/train.csv"
urlretrieve(url,"train.csv")
url = "https://github.com/hoiyurie/tibame-ml0602/raw/main/titanic/test.csv"
urlretrieve(url,"test.csv")


('test.csv', <http.client.HTTPMessage at 0x7f3313637510>)

In [19]:
import pandas as pd
train_df = pd.read_csv("train.csv",encoding="utf-8")
test_df = pd.read_csv("test.csv",encoding="utf-8")

In [20]:
# 先把train data 和test data 兩個表格連結起來
data = pd.concat([train_df, test_df],ignore_index = True) # ingnore_index = True 重置合併後的index
data = data.drop(["PassengerId","Survived"], axis = 1)
# data

In [21]:
# 查看空值 data.isna（）
# 查看空值總數 data.isna().sum()
# 對Series做篩選[帶入跟你的資料筆數一樣多True/False list]

na = data.isna().sum()
na[na>0].sort_values(ascending= False) # ascending = False 小-大



Cabin       1014
Age          263
Embarked       2
Fare           1
dtype: int64

In [5]:
# # apply 

# s = pd.Series([1,2,3])
# def func(n):
#   return n * n 
# s.apply(func)


# # lambda = def 
# # 多行 def
# # 少數幾行 lambda

# s= pd.Series([1, 2, 3])
# def func(n):
#   return n * n
# s.apply(lambda n:n*3)


# # 可以直接寫成
# s= pd.Series([1, 2, 3])
# s.apply(lambda n:n*3)


In [23]:
# 先處理一些數據：

#Cabin 取出首字母，可能代表艙位等級

def cabin_head(s):
  if not pd.isna(s):
    return s[0]
  # else:
  #   return s  # 回傳NAN

data["Cabin"] = data["Cabin"].apply(cabin_head)


In [24]:
# Ticket 可以看出同行人數

dic = data["Ticket"].value_counts()
data["Ticket"] = data["Ticket"].apply(lambda t:dic[t])

In [25]:
# 補缺失值（類別：最常出現）
# ！！！如果有一堆測試資料，不要重新算，直接補S(補最多的) !!!! 

# Embarked  登船點--> 補成 最多人使用的登船點
# data["Embarked"]

most = data["Embarked"].value_counts().idxmax() # idxmax 最大值的索引
data["Embarked"] = data["Embarked"].fillna(most)
na = data.isna().sum()
na[na > 0].sort_values(ascending = False)


Cabin    1014
Age       263
Fare        1
dtype: int64

In [26]:
# 補缺失值（數值：中位數）

med = data.median().drop(["Pclass"]) # Pclass 不能補中位數, 先除掉
data = data.fillna(med)
na = data.isna().sum()
na[na > 0].sort_values(ascending = False)

Cabin    1014
dtype: int64

In [10]:
# data --> 看名字


In [27]:
# Braund, Mr. Owen Harris

def name_convert(s):
  s = s.split(",")[-1].split(".")[0] # 把Braund, Mr. Owen Harris 切割，取出Mr
  s= s.strip() # 去掉前後空格
  return s 
counts = data["Name"].apply(name_convert).value_counts()
whitelist = counts[counts > 50].index
# whitelist
def name_convert(s):
  s = s.split(",")[-1].split(".")[0]
  s = s.strip()
  if s in whitelist:
    return s
  else:
    return None
  
data["Name"] = data["Name"].apply(name_convert)


In [28]:
# One-Hot encoding

data = pd.get_dummies(data)
data = pd.get_dummies(data, columns=["Pclass"])
data


# 特別針對Pclass再做一次one-hot



,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3
0,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,38.0,1,0,2,71.2833,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
2,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
3,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0
4,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1


In [29]:
# 模型建立完, 後續的嘗試, 只是我嘗試過, 所以我直接做
data["Family"] = data["SibSp"] + data["Parch"]
data

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Family
0,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1
1,38.0,1,0,2,71.2833,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1
2,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1
4,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [ ]:
#### RandomForest


In [30]:
print(len(data))
print(data.shape)

1309
(1309, 26)


In [31]:
# .loc(根據列編號)（X)   
# .iloc(根據第幾個)(O)
# .iloc[第一列，第二列，第三列...]

x_train = data.iloc[:train_df.shape[0]]
y_train = train_df["Survived"]
x_predict = data.iloc[train_df.shape[0]:]
# x_train


In [32]:
data

,Age,SibSp,Parch,Ticket,Fare,Name_Master,Name_Miss,Name_Mr,Name_Mrs,Sex_female,Sex_male,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,Cabin_T,Embarked_C,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Family
0,22.0,1,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1
1,38.0,1,0,2,71.2833,0,0,0,1,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1
2,26.0,0,0,1,7.9250,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
3,35.0,1,0,2,53.1000,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,0,1
4,35.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1304,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1305,39.0,0,0,3,108.9000,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0
1306,38.5,0,0,1,7.2500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0
1307,28.0,0,0,1,8.0500,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0


In [ ]:

import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
params = {
    # 1. 5 2. [1, 2, 3] 3. range
    # 20~99
    "n_estimators":range(20, 100),
    # 3~10
    "max_depth":range(3, 11)
}
clf = RandomForestClassifier()
cv = GridSearchCV(clf, params, cv=10, n_jobs=-1)
cv.fit(x_train, y_train)
print(cv.best_score_)
print(cv.best_params_)

# RandomForest 可以 classifier/regressor/embedding 

In [ ]:
clf = RandomForestClassifier(n_estimators=25, max_depth=6) # n_estimators 隨機森林的數量（20~100）
cross_val_score（clf, x_train, cv = 10, n_jobs = -1) # cv

